# Nodding

This notebook shows the reduction path of a nodding observation in a multi-beam receiver.

Similar to a position-switched observation, two beams are selected for simultaneous observing (though the receiver can have more than two beams). In the first scan BEAM1 is looking at the source, while BEAM2 is looking at an assumed OFF position. In the next scan, BEAM2 will be looking at the source, while BEAM1 is looking at (another) OFF position. This will result in two position-switched solutions, which are then averaged for the final spectrum. 

The obvious big advantage of this observing mode is that the telescope is always ON source, bringing the noise down by $\sqrt{2}$, minus a small amount of slewing time.

This notebook also includes the matching getps() observation from another scan. 


In [ ]:
# %matplotlib widget

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from dysh.fits.gbtfitsload import GBTFITSLoad
from dysh.util.files import dysh_data

In [ ]:
path = "/home/dysh/acceptance_testing/data/"
path = "/home/teuben/GBT/dysh_data/example_data"
file1 = f"{path}/nod-KFPA/data/TGBT22A_503_02.raw.vegas"
print(file1)

In [ ]:
file1 = dysh_data(example="nod-KFPA/data/TGBT22A_503_02.raw.vegas")

This is a huge dataset (19GB) and can take some time to load if it's the first time.

In [ ]:
sdf = GBTFITSLoad(file1)

In [ ]:
sdf.summary()

We will use scans 60 and 61 for the position-switching (PS) observation, and average the two polarizations.
Technically the averaging should be done weighthing by 1/TSYS**2, but we are lazy here.

Also note for the 7-beam KFPA receiver, the central beam (fdnum=0) will be the source tracking beam.

In [ ]:
sp0 = sdf.getps(scan=60,plnum=0,ifnum=0,fdnum=0).timeaverage().smooth('box',11)

In [ ]:
sp0.plot(title='dysh getps',xaxis_unit="chan")

Now the nodding method, using scans 62 and 63.  This will return two scanblocks, representing the two "PS" observations that need to be averaged to get the final spectrum.  We can do that with the timeaverage() function directly applying to this list of (two) scanblocks.  And for good measure we will smooth it a little bit as we did with the PS observation.

Normally the FITS header will contain information on which is BEAM1 and which is BEAM2. 

In [ ]:
sp2 = sdf.getnod(scan=[62,63],ifnum=0,plnum=0).timeaverage().smooth('box',11)

In [ ]:
sp2.plot(title='dysh getnod',xaxis_unit="chan")
print('getnod:', sp2[200:1200].stats()['rms'])
print('getps: ', sp0[200:1200].stats()['rms'])
